In [25]:
import numpy as np
from sklearn.manifold import TSNE
import pickle
# import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)
import time
import sqlite3
import pandas as pd
import json
import colorlover as cl

In [2]:
song_id_path = "./data/song_track_artist.pkl"

# Get popularity embedding latent vectors

In [76]:
popularity_embeddings_path = "./data/popularity_latent_embeddings.npy"
track_id_embeddings_path = "./data/track_ids.txt"

In [77]:
popularity_embeddings_np = np.load(popularity_embeddings_path)
track_ids = np.loadtxt(track_id_embeddings_path, dtype="str")
popularity_embeddings_duplicates = pd.DataFrame(popularity_embeddings_np, columns=list(range(popularity_embeddings_np.shape[1])))
popularity_embeddings_duplicates.index = track_ids


In [78]:
popularity_embeddings = popularity_embeddings_duplicates[~popularity_embeddings_duplicates.index.duplicated(keep='first')]

print(popularity_embeddings.shape)
print(popularity_embeddings.head())


(32648, 50)
                          0         1    2         3         4         5   \
TRMMMTK128F424EF7C  0.041793  0.000071  0.0  0.000067  0.034190  0.162911   
TRMMMKQ128F92EBCB5  0.054502  0.000078  0.0  0.000085  0.051364  0.165174   
TRMMMFG128F425087B  0.021546  0.000063  0.0  0.000057  0.031385  0.203347   
TRMMMQA128F14A454A  0.004724  0.000034  0.0  0.000012  0.006738  0.066585   
TRMMWJS12903CBB7F5  0.031037  0.000064  0.0  0.000052  0.021773  0.139371   

                          6    7         8         9   ...        40  \
TRMMMTK128F424EF7C  0.010294  0.0  0.236780  0.157144  ...  0.000000   
TRMMMKQ128F92EBCB5  0.005222  0.0  0.324334  0.214856  ...  0.000000   
TRMMMFG128F425087B  0.005848  0.0  0.163397  0.106788  ...  0.000000   
TRMMMQA128F14A454A  0.002401  0.0  0.047468  0.028869  ...  0.015307   
TRMMWJS12903CBB7F5  0.007585  0.0  0.189007  0.125123  ...  0.000000   

                          41        42        43   44        45        46  \
TRMMMTK128F424E

# get genre

In [79]:
with open("./data/track-genre-mapping.json", "r") as f:
    track_genre = json.load(f)

# get popularity

In [80]:
import pickle

with open("data/nn_subset_popularity.pkl", "rb") as f:
    popularity_data = pickle.load(f)
print(popularity_data.shape)
popularity_data.head()

(32648, 1)


,play_count
track_id,
TRMMMTK128F424EF7C,332
TRMMMKQ128F92EBCB5,128
TRMMMFG128F425087B,690
TRMMMQA128F14A454A,157
TRMMWJS12903CBB7F5,221


# Set track id filter to filtered_track_ids (used throughout)

In [81]:
filtered_track_ids = popularity_embeddings.index[:3000]

# Get song_id to song_name mapping


### using pkl file

In [82]:
with open(song_id_path, "rb") as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    song_name_artist = u.load()

In [83]:
# getting everything in the right order
song_name_artist_ordered = song_name_artist.set_index(["track_id"]).loc[filtered_track_ids]
# song_artist_str = song_name_artist_ordered.apply(lambda r: "artist: " + r["artist_id"] + ", song: " + r["song_name"], axis=1)
song_artist_str = song_name_artist_ordered["artist_id"]




In [84]:
song_name_artist_ordered.shape

(3000, 3)

In [85]:
artists = song_name_artist_ordered["artist_id"].values

# Mark artists

In [86]:
famous = ["Lady GaGa", "Kanye West", "Bob Marley", "Bob Dylan", "Shakira", "Jay-Z", "Bon Jovi", "Michael Jackson", 
         "Mariah Carey", "Justin Timberlake", "Oasis"]

In [87]:
# colors = [int(a in famous) for a in artists]

In [88]:
# max_popularity = song_popularity.iloc[0].values[0]
# pre_colors = np.array([x[0]/max_popularity for x in song_popularity.values])

In [89]:
genres = list(set(track_genre.values()))
genres

['Pop_Rock',
 'Metal',
 'Pop',
 'Reggae',
 'Punk',
 'World',
 'Country',
 'Latin',
 'Electronic',
 'Blues',
 'Rock',
 'Vocal',
 'International',
 'Rap',
 'Jazz',
 'RnB',
 'Folk',
 'New']

In [90]:
genres.index("Blues")

9

In [91]:
def foo_genre(t):
    global genres
    if t in track_genre:
        return genres.index(track_genre[t])
    return -1
colors = song_name_artist_ordered.index.map(foo).values

In [92]:
bins =  [124. ,  156. ,  198. ,  258.  , 336.  , 451.  , 622.   ,891.9 ,1430.6, 2792.3]
def foo_pop(t):
    global bins
    pop = popularity_data.loc[t]["play_count"]
    for i in range(1, len(bins)):
        if pop < bins[i]:
            return i
    return 10
    
    
colors = song_name_artist_ordered.index.map(foo_pop).values

# Create t-SNE

In [93]:
filtered_track_ids

Index(['TRMMMTK128F424EF7C', 'TRMMMKQ128F92EBCB5', 'TRMMMFG128F425087B',
       'TRMMMQA128F14A454A', 'TRMMWJS12903CBB7F5', 'TRMMWQU12903CF447F',
       'TRMMGTX128F92FB4D9', 'TRMMHDI128F42442F4', 'TRMMHKG12903CDB1B5',
       'TRMMCBU128E0791823',
       ...
       'TRGIJKQ128F930020D', 'TRGIVGF128F92FE01C', 'TRGIVZT128F42776BD',
       'TRGIVBQ12903CCEAD9', 'TRGIDGJ128F9351E52', 'TRGIDEJ128EF349E58',
       'TRGIOWH128F426A72F', 'TRGIXJH128F1452951', 'TRGIYDK128F423EEB0',
       'TRGIYIR128F92DCDF1'],
      dtype='object', length=3000)

In [94]:
embeddings_pure = popularity_embeddings.loc[filtered_track_ids].values

In [95]:
embeddings_pure.shape

(3000, 50)

In [96]:
X_embedded = TSNE(n_components=2).fit_transform(embeddings_pure)
X_embedded.shape

(3000, 2)

In [97]:
by_genre_factors = {i: X_embedded[i+1 == colors[:len(X_embedded)]] for i in range(len(bins))}
by_genre_text = {i: song_artist_str[i+1 == colors] for i in range(len(bins))}

In [98]:
for k, v in by_genre_factors.items():
    print(v.shape)

(304, 2)
(294, 2)
(302, 2)
(274, 2)
(321, 2)
(312, 2)
(301, 2)
(291, 2)
(308, 2)
(293, 2)


In [99]:
bupu = cl.scales['11']['div']['RdYlBu']

bupu500 = cl.interp( bupu, 18 )

# create a new plot with a title and axis labels
scale = [[0.0, 'rgb(165,0,38)'], [0.1111111111111111, 'rgb(215,48,39)'], [0.2222222222222222, 'rgb(244,109,67)'],
        [0.3333333333333333, 'rgb(253,174,97)'], [0.4444444444444444, 'rgb(254,224,144)'], [0.5555555555555556, 'rgb(224,243,248)'],
        [0.6666666666666666, 'rgb(171,217,233)'],[0.7777777777777778, 'rgb(116,173,209)'], [0.8888888888888888, 'rgb(69,117,180)'],
        [1.0, 'rgb(49,54,149)']]

data = [go.Scatter(x=by_genre_factors[i][:,0], y=by_genre_factors[i][:,1], mode='markers',
    marker=dict(
        size=7,
        color=bupu500[i]
    ), text=np.array(by_genre_text[i]), name=str(i+1)) for i in range(len(bins))]

# fig = {
#     'data': [
#         {
#             'x': df[df['year']==year]['gdpPercap'],
#             'y': df[df['year']==year]['lifeExp'],
#             'name': year, 'mode': 'markers',
#         } for year in [1952, 1982, 2007]
#     ],
#     'layout': {
#         'xaxis': {'title': 'GDP per Capita', 'type': 'log'},
#         'yaxis': {'title': "Life Expectancy"}
#     }
# }


plotly.offline.iplot(data, filename="tsne-scatter")